In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import keras
import tensorflow
import math
import gc
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['train.csv', 'sample_submission.csv', 'test.csv']


In [7]:

gc.collect()

0

In [8]:
dtypes = {
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'OsVer':                                                'category',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'Census_MDC2FormFactor':                                'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_FlightRing':                                    'category',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }


In [9]:
train=pd.read_csv('../input/train.csv',usecols=list(dtypes.keys()))
train.shape
#train = pd.read_csv('../input/train.csv',usecols=list(dtypes.keys()), usecols = [])

(8921483, 59)

In [10]:
trans_dict = {
    'off': 'Off', '&#x02;': '2', '&#x01;': '1', 'on': 'On', 'requireadmin': 'RequireAdmin', 'OFF': 'Off', 
    'Promt': 'Prompt', 'requireAdmin': 'RequireAdmin', 'prompt': 'Prompt', 'warn': 'Warn', 
    '00000000': '0', '&#x03;': '3', np.nan: 'ExistsNotSet'
}
train.replace({'SmartScreen': trans_dict}, inplace=True)

In [11]:
trans_dict = {
    'Off':'other', '2':'other',  '1':'other',  'On':'other', 'requireadmin': 'RequireAdmin',  'Off':'other', 
    'Prompt':'other', 'requireAdmin': 'RequireAdmin','Warn':'other', 
    '0':'other', '3':'other', np.nan: 'ExistsNotSet', 'Enabled':'other',"Block":'other'
}
train.replace({'SmartScreen': trans_dict}, inplace=True)

In [12]:
train.OrganizationIdentifier=train.OrganizationIdentifier.fillna(0)

In [13]:
train.Census_InternalBatteryNumberOfCharges=train.Census_InternalBatteryNumberOfCharges.fillna(-5)

In [14]:
train.dropna(inplace=True)
train.shape

(7835830, 59)

In [15]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        print(col_type)
        if col_type != ('object' or 'category'):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    gc.collect()
    return df

%time
#train = reduce_mem_usage(train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.2 µs


In [16]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 3586.96 MB
object
object
object
float64
int64
float64
float64
float64
int64
int64
float64
float64
float64
int64
object
int64
object
object
object
float64
float64
object
float64
object
float64
float64
float64
float64
float64
object
float64
int64
float64
object
float64
float64
object
float64
object
object
object
int64
int64
object
object
int64
object
object
object
object
float64
float64
int64
int64
int64
float64
float64
float64
int64
Memory usage after optimization is: 881.86 MB
Decreased by 75.4%


In [17]:
gc.collect()

0

In [18]:
low_cardinality_cols = train.select_dtypes(include=['category','object']).columns

In [19]:
low_cardinality_cols
len(low_cardinality_cols)

21

In [20]:
cat_col_rem=[]
for colm in low_cardinality_cols:
    x=set(train[colm])
    if len(x)==1:
        train.drop(colm)
        pritn("x")
    elif len(x)==2:
        train[colm].astype('bool')
        pritn("x")
    elif len(x)>=6:
        cat_col_rem.append(colm)

In [21]:
cat_col_rem.remove('OsPlatformSubRelease')
cat_col_rem.remove('SkuEdition')
cat_col_rem.remove('Census_OSInstallTypeName')
print(cat_col_rem)

['EngineVersion', 'AppVersion', 'OsVer', 'OsBuildLab', 'Census_MDC2FormFactor', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_OSVersion', 'Census_OSBranch', 'Census_OSEdition', 'Census_FlightRing']


In [ ]:
change_dict = {'ProductName':-3,
               'EngineVersion':-6,
               'AppVersion':-9, 
               'OsVer':0,
               'OsPlatformSubRelease':-7, 
               'OsBuildLab':-8, 
               'SkuEdition':-5, 
               'SmartScreen':0,
               'Census_MDC2FormFactor':-4, 
               'Census_PrimaryDiskTypeName':-2,
               'Census_ChassisTypeName':-4,
               'Census_PowerPlatformRoleName':-2, 
               'Census_OSVersion':-9, 
               'Census_OSArchitecture':0, 
               'Census_OSBranch':-7, 
               'Census_OSEdition':-3, 
               'Census_OSInstallTypeName':-7,
               'Census_OSWUAutoUpdateOptionsName':-3, 
               'Census_GenuineStateName':-3, 
               'Census_ActivationChannel':-3,
               'Census_FlightRing':-2}

In [22]:
#print(len(low_cardinality_cols))
#print(len(cat_col_rem),cat_col_rem)
#AvSigVersion
#'Census_OSInstallTypeName' , 'SkuEdition' , 'OsPlatformSubRelease'
try:   
    for colx in cat_col_rem[0:1]:
        print(colx)
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    print("sfsdad")
    for colx in cat_col_rem[0:1]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)



EngineVersion


In [23]:
try:    
    for colx in cat_col_rem[1:2]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[1:2]: 
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


14

In [24]:
try:    
    for colx in cat_col_rem[2:3]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[2:3]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [25]:
try:    
    for colx in cat_col_rem[3:4]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[3:4]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})
        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [26]:
try:    
    for colx in cat_col_rem[4:5]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[4:5]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [27]:
try:    
    for colx in cat_col_rem[5:6]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['Other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[5:6]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] +=k.pop(_)
                trans_dict.update({_:'other'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [28]:
try:    
    for colx in cat_col_rem[6:7]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[6:7]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [29]:
try:    
    for colx in cat_col_rem[7:8]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[7:8]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [30]:
try:    
    for colx in cat_col_rem[8:9]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[8:9]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [31]:
try:    
    for colx in cat_col_rem[9:10]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[9:10]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [32]:
try:    
    for colx in cat_col_rem[10:11]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[10:11]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})


        #print("k = ",trans_dict)
train.replace({colx: trans_dict}, inplace=True)
gc.collect()


7

In [33]:
try:    
    for colx in cat_col_rem[11:12]:
        k = {}
        k = train[colx].value_counts().to_dict()
        k.update({'other':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other'] += k.pop(_)
                trans_dict.update({_:'Other'})
except:
    for colx in cat_col_rem[11:12]:
        k={}
        k = train[colx].value_counts().to_dict()
        k.update({'other2':0})
        trans_dict = {}
        for _ in list(k.keys()):
            if k[_] <= 100000:
                k['other2'] +=k.pop(_)
                trans_dict.update({_:'other2'})

        #print("k = ",trans_dict)   
train.replace({colx: trans_dict}, inplace=True)


In [34]:
gc.collect()

0

In [35]:
gc.collect()
for _ in cat_col_rem:
    print(train[_].value_counts())
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

1.1.15200.1    3346534
1.1.15100.1    3304118
Other           422362
1.1.15000.2     239281
1.1.14901.4     191641
1.1.14800.3     122631
1.1.14600.4     105821
1.1.15300.6     103442
Name: EngineVersion, dtype: int64
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
4.18.1807.18075     4632528
4.18.1806.18062      771472
Other                734200
4.12.16299.15        291862
4.16.17656.18052     216564
4.13.17134.228       187597
4.9.10586.1106       182330
4.8.10240.17443      181572
4.13.17134.1         179678
4.14.17639.18041     178737
4.10.209.0           170761
4.12.17007.18022     108529
Name: AppVersion, dtype: int64
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
10.0.0.0    7660441
6.3.0.0      163267
Other         12122
Name: OsVer, dtype: int64
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
17134.1.amd64fre.rs4_release.180410-1804                 3235381
16299.431.amd64fre.rs3_release_svc_escrow.180502-1908    1154292
Other                                                    1079567

lot of values:EngineVersion
             :AppVersion
             :OsVer     10.0.0.0 ,other
             :OsBuildLab
             :Census_OSVersion
             :Census_OSBranch
             :Census_OSEdition
             
restructure  :SmartScreen -- Done
             :Census_ChassisTypeName

other        :Census_MDC2FormFactor

In [36]:
train['SkuEdition'].value_counts()

Home               4984108
Pro                2756062
Education            36759
Enterprise           28565
Enterprise LTSB      16938
Invalid               8465
Cloud                 4933
Name: SkuEdition, dtype: int64

In [ ]:
cat_col_rem=[]
for colm in low_cardinality_cols:
    x=set(train[colm])
    if len(x)==1:
        train.drop(colm)
        pritn("x")
    elif len(x)==2:
        train[colm].astype('bool')
        pritn("x")
    elif len(x)>=3:
        cat_col_rem.append(colm)
print(cat_col_rem)

In [ ]:
i=0
for colx in cat_col_rem:
    #plot_count = sns.countplot(y="HasDetections", hue="which", data = train[colx])
    plt.figure(i,figsize=(50,50))
    plt.hist([train[colx],train['HasDetections']])
    plt.title(colx)
    plt.show()
    i+=1

{'ProductName':3, 'EngineVersion':6, 'AppVersion':9, 'OsVer':0, 'OsPlatformSubRelease':7, 'OsBuildLab':8, 'SkuEdition':5, 'SmartScreen':0, 'Census_MDC2FormFactor':4, 'Census_PrimaryDiskTypeName'2, 'Census_ChassisTypeName':4, 'Census_PowerPlatformRoleName':2, 'Census_OSVersion':9, 'Census_OSArchitecture':0, 'Census_OSBranch':7, 'Census_OSEdition':3, 'Census_OSInstallTypeName':7, 'Census_OSWUAutoUpdateOptionsName':3, 'Census_GenuineStateName':3, 'Census_ActivationChannel':3, 'Census_FlightRing':2}

In [37]:
train = pd.get_dummies(train)

In [38]:
train.head()

,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsSuite,IsProtected,IeVerIdentifier,Firewall,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSUILocaleIdentifier,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections,ProductName_mse,ProductName_mseprerelease,...,Census_OSBranch_rs2_release,Census_OSBranch_rs3_release,Census_OSBranch_rs3_release_svc_escrow,Census_OSBranch_rs4_release,Census_OSBranch_th1_st1,Census_OSBranch_th2_release,Census_OSBranch_th2_release_sec,Census_OSEdition_Core,Census_OSEdition_CoreCountrySpecific,Census_OSEdition_CoreSingleLanguage,Census_OSEdition_Other,Census_OSEdition_Professional,Census_OSInstallTypeName_Clean,Census_OSInstallTypeName_CleanPCRefresh,Census_OSInstallTypeName_IBSClean,Census_OSInstallTypeName_Other,Census_OSInstallTypeName_Refresh,Census_OSInstallTypeName_Reset,Census_OSInstallTypeName_UUPUpgrade,Census_OSInstallTypeName_Update,Census_OSInstallTypeName_Upgrade,Census_OSWUAutoUpdateOptionsName_AutoInstallAndRebootAtMaintenanceTime,Census_OSWUAutoUpdateOptionsName_FullAuto,Census_OSWUAutoUpdateOptionsName_Notify,Census_OSWUAutoUpdateOptionsName_Off,Census_OSWUAutoUpdateOptionsName_UNKNOWN,Census_GenuineStateName_INVALID_LICENSE,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_TAMPERED,Census_GenuineStateName_UNKNOWN,Census_ActivationChannel_OEM:DM,Census_ActivationChannel_OEM:NONSLP,Census_ActivationChannel_Retail,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK,Census_FlightRing_NOT_SET,Census_FlightRing_Other,Census_FlightRing_Retail,Census_FlightRing_Unknown
0,7.0,0,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,256,1.0,137.0,1.0,2668.0,9124.0,4.0,2340.0,476940.0,299451.0,0,4096.0,18.906250,1440.0,4.294967e+09,17134,165,119,628.0,36144.0,0,0,0,0.0,0.0,10.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0
1,7.0,0,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,256,1.0,137.0,1.0,2668.0,91656.0,4.0,2404.0,476940.0,102385.0,0,4096.0,13.898438,1366.0,1.000000e+00,17134,1,31,628.0,57858.0,0,0,0,0.0,0.0,8.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0
2,7.0,0,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,768,1.0,137.0,1.0,4908.0,317701.0,4.0,1972.0,114473.0,113907.0,0,4096.0,21.500000,1920.0,4.294967e+09,17134,165,30,142.0,52682.0,0,0,0,0.0,0.0,3.0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
3,7.0,0,53447.0,1.0,1.0,1,88,20710.0,0.0,117.0,115,256,1.0,137.0,1.0,1443.0,275890.0,4.0,2272.0,238475.0,227116.0,0,4096.0,18.500000,1366.0,4.294967e+09,17134,228,64,355.0,20050.0,0,0,0,0.0,0.0,3.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
4,7.0,0,53447.0,1.0,1.0,1,18,37376.0,0.0,277.0,75,768,1.0,137.0,1.0,1443.0,331929.0,4.0,2500.0,476940.0,101900.0,0,6144.0,14.000000,1366.0,0.000000e+00,17134,191,31,355.0,19844.0,0,0,0,0.0,0.0,1.0,1,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0


In [39]:
train.shape

(7835830, 177)

In [ ]:
#train_output=train['HasDetections']
#train=train.drop('HasDetections',axis=1)
#train.shape

In [40]:
train.to_csv('train_proc.csv')

In [42]:
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
#clf=MLPClassifier(hidden_layer_sizes=(65,), alpha=0.05)

In [47]:
k = 0
for train in pd.read_csv("train_proc.csv",chunksize = 2000000, dtype = dtypes):
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    clf.partial_fit(train[list(train)[0:-1]],train[list(train)[-1]],classes=np.array([0,1]))
    print("Selected 2000000 rows for training")
    k+=1
    if k==3:
        break

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Selected 2000000 rows for training
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Selected 2000000 rows for training
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Selected 2000000 rows for training


In [53]:
test = pd.read_csv("train_proc.csv", dtype = dtypes, skiprows = range(0,6000000))
op = clf.predict(test[list(test)[0:-1]])
accuracy = sklearn.metrics.accuracy_score(test[list(test)[-1]],op)
print(accuracy)

0.9743396719739845


In [41]:
!ls


__notebook_source__.ipynb  train_proc.csv


In [48]:
del train

In [52]:
del test

NameError: name 'test' is not defined

In [49]:
gc.collect()

299

In [5]:
test.shape

NameError: name 'pd' is not defined

{}

In [ ]:
cols = list(train.columns.values)
plt.figure(figsize=(100,100))
co_cols = cols[:]
sns.heatmap(train[co_cols].corr(), cmap='RdBu_r', annot=True, center=0.0)
plt.title('Correlation between 1 ~ 10th columns')
plt.show()

In [ ]:
train = reduce_mem_usage(train)

In [ ]:
from IPython.display import HTML
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
create_download_link(train)

In [ ]:
train.isna().any()

In [ ]:
clf.partial_fit(train[list(train)[0:-1]],train[list(train)[-1]],classes=np.array([0,1]))

In [ ]:
"""Label encoder code"""
'''
train['SmartScreen'] = train.SmartScreen.astype('category')
#train['Census_InternalBatteryType'] = train.Census_InternalBatteryType.astype('category')

cate_cols = train.select_dtypes(include=['category','object']).columns.tolist()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cate_cols:
    train[col] = le.fit_transform(train[col])
'''